In [5]:
!pip install pyyaml==5.1
!pip install "git+https://github.com/facebookresearch/detectron2.git"

  Cloning https://github.com/facebookresearch/detectron2.git to c:\users\dummy\appdata\local\temp\pip-req-build-35jzpvbo
  Resolved https://github.com/facebookresearch/detectron2.git to commit 3cfda4704cb3b23255f29d683cf05cbaf10daaa4
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached future-0.18.2-py3-none-any.whl
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started


  Running command git clone -q https://github.com/facebookresearch/detectron2.git 'C:\Users\dummy\AppData\Local\Temp\pip-req-build-35jzpvbo'


In [6]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

torch:  1.10 ; cuda:  1.10.1
detectron2: 0.6


'nvcc' is not recognized as an internal or external command,
operable program or batch file.


Import Detectron

In [8]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
import cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
from detectron2.data.datasets import register_coco_instances
data_path = "/content/drive/MyDrive/PHD/Internships/Pittsburg University - Xing Lab/Collab_datatrain.json","/content/drive/MyDrive/PHD/Internships/Pittsburg University - Xing Lab/Collab_data/data"
val_data_path = "/content/drive/MyDrive/PHD/Internships/Pittsburg University - Xing Lab/Collab_dataval.json","/content/drive/MyDrive/PHD/Internships/Pittsburg University - Xing Lab/Collab_data/data"
register_coco_instances("deepfashion_train", {}, data_path)
register_coco_instances("deepfashion_val", {}, val_data_path)

## Detectron config

In [9]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("deepfashion_val",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.MAX_ITER = 300
cfg.SOLVER.STEPS = (1000, 1500)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 64
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13

cfg.TEST.EVAL_PERIOD = 500